<a href="https://colab.research.google.com/github/darioblandon/materials-fa23/blob/main/lab/lab10/lab10_td.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab10.ipynb")

In [ ]:
# Run this cell to set up the notebook, but please don't change it.

# These lines import the Numpy and Datascience modules.
import numpy as np
from datascience import *

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore')

# This line loads the visualization code for this lab.
import visualizations

# 1. ¿Para qué sirve la probabilidad condicional?

Supongamos que tenemos una población conocida, como todos los perros de California. Hasta ahora, hemos visto tres formas de *predecir* algo sobre un individuo en esa población, dado un conocimiento incompleto sobre la identidad del individuo:

* Si no sabemos nada sobre el perro individual, podríamos predecir que su velocidad es el *promedio* o *mediana* de todas las velocidades de la población.
* Si conocemos la altura del perro pero no su velocidad, podríamos usar *regresión lineal* para predecir su velocidad a partir de su altura. La predicción resultante sigue siendo imperfecta, pero podría ser más precisa que el promedio de la población.
* Si conocemos la raza, la altura y la edad del perro, podríamos usar la *clasificación del vecino más cercano* para predecir su velocidad comparándolo con un grupo de perros con velocidad conocida.


También podemos calcular probabilidades condicionales para hacer predicciones sobre individuos o eventos. Esta técnica es diferente de los métodos anteriores que hemos examinado porque

1. nuestra predicción para cada resultado se describe mediante una probabilidad, y
2. cada probabilidad puede calcularse exactamente a partir de suposiciones, en lugar de estimarse a partir de datos.

# 2. Matrices de iconos
Las partes 3 y 4 de esta práctica de laboratorio trabajan con un ejemplo más complejo sobre enfermedades, pero primero, comencemos con un ejemplo simple.

Imagina que Samantha y tú estáis jugando un juego en el que os dan una canica y os piden que determines su textura y tamaño. No sabes nada sobre la canica que te dan, pero sabes que Samantha la sacó **uniformemente al azar** de una bolsa que contenía las siguientes canicas:
* 4 canicas grandes y brillantes,
* 1 canica grande sin brillo,
* 6 pequeñas canicas brillantes,
* 2 canicas pequeñas sin brillo.

**Pregunta 2.0.1.** Sabiendo sólo lo que te hemos dicho hasta ahora, ¿cuál es la probabilidad de que la canica que te dieron sea una canica grande y brillante?


In [ ]:
probability_large_shiny = ...

In [ ]:
grader.check("q2_0_1")

Here's a table with those marbles:

In [ ]:
marbles = Table.read_table("marbles.csv")
marbles.show()

Aquí están los recuentos de cada tipo de canica en una tabla dinámica.

In [ ]:
marbles.pivot('surface', 'size')

Aquí están todas las diferentes combinaciones de superficie y tamaño, con el recuento de cada combinación de superficie y tamaño. Cada tipo de mármol aparece en su propia fila.

In [ ]:
marbles.group(['surface', 'size'])

Hemos incluido código para mostrar algo llamado *matriz de iconos*. Las funciones de la celda siguiente crean matrices de iconos a partir de varios tipos de tablas. No te preocupes por entender el código; simplemente ejecuta esta celda.

**NOTA:** Puede ignorar las etiquetas del eje y. Solo recuerda que cada caja representa 1 canica.

In [ ]:
# Ejecute esta celda.

#######################################################################
# Las funciones que necesitarás utilizar están aquí. Cada uno es un
# forma de crear una matriz de iconos a partir de una tabla con formato diferente.
#######################################################################

def display_icon_array(table, groups, individuals_name):
    """
     Dada una tabla y algunas columnas para agruparla, muestra una matriz de iconos
     de los grupos.

     Los grupos deben ser una serie de etiquetas de columnas en la tabla.

     individuos_name es su nombre para las filas individuales de la tabla.
     Por ejemplo, si hablamos de una población de personas,
     individuos_name debe ser "personas".

     Por ejemplo:

     display_icon_array(canicas, ["superficie", "tamaño"], "canicas")
     """
    display_grouped_icon_array(table.group(groups), individuals_name)

def display_grouped_icon_array(grouped_data, individuals_name):
    """
     Dada una tabla con recuentos de datos agrupados por 1 o más categorías,
     muestra una matriz de iconos de los grupos representados en la tabla.

     grouped_data debe ser una tabla de frecuencias o recuentos, como
     una tabla creada llamando al método de grupos en alguna tabla.

     individuos_name es su nombre para los miembros individuales del
     conjunto de datos. Por ejemplo, si hablamos de una población de
     personas, nombre_individuos debe ser "personas".

     Por ejemplo:

     display_grouped_icon_array(canicas.group(["superficie", "tamaño"]), "canicas")
    """
    visualizations.display_combinations(grouped_data, individuals_name=individuals_name)

def display_crosstab_icon_array(crosstabulation, x_label, individuals_name):
    """
     Dada una tabla de tabulación cruzada, muestra una matriz de iconos de los grupos
     representado en la tabla.

     La tabulación cruzada debe ser una tabla de frecuencias o recuentos creada por
     llamando a pivote en alguna mesa.

     x_label debe ser la etiqueta de las categorías enumeradas como columnas (en
     el "eje x" cuando se imprime la tabla de contingencia).

     individuos_name es su nombre para los miembros individuales del
     conjunto de datos. Por ejemplo, si hablamos de una población de
     personas, nombre_individuos debe ser "personas".

     For example:
     display_crosstab_icon_array(marbles.pivot("surface", "size"), "surface", "marbles")
    """
    display_grouped_icon_array(visualizations.pivot_table_to_groups(crosstabulation, x_label), individuals_name)

Aquí hay una serie de iconos de todas las canicas, agrupadas por superficie y tamaño:

In [ ]:
# Run this cell.
display_grouped_icon_array(marbles.group(["surface", "size"]), "marble(s)")

Debes imaginar que la canica que te dieron fue seleccionada mediante un sorteo aleatorio entre estos 13 íconos.

La siguiente es una serie de íconos de las canicas, agrupadas **solo por su superficie (brillante/opaca)**.

In [ ]:
display_grouped_icon_array(marbles.group("surface"), "marble(s)")

Sin saber nada más sobre el mármol, es igualmente probable que sea cualquiera de los mármoles representados arriba; Esto se debe a que hemos asumido que la canica se selecciona **uniformemente al azar** de la bolsa.

**Pregunta 2.0.2.** ¿Cuál es la probabilidad de que te hayan dado una canica brillante? Responda usando expresiones matemáticas mirando su conjunto de iconos.


In [ ]:
probability_shiny = ...

In [ ]:
grader.check("q2_0_2")

## 2.1. La probabilidad condicional

Supongamos que escuchas a Samantha decir que te dieron una canica grande. ¿Esto cambia de alguna manera la posibilidad de que tu canica brille? Vamos a averiguar.

Vuelva a la matriz de iconos completa, que se muestra a continuación para mayor comodidad.

In [ ]:
display_grouped_icon_array(marbles.group(["surface", "size"]), "marble(s)")

En la pregunta 2.0.2, asumimos que la canica que te dieron tenía la misma probabilidad de ser cualquiera de las canicas, simplemente porque no sabíamos nada mejor. Por eso miramos todas las canicas para calcular la probabilidad de que tu canica fuera brillante.

Pero suponiendo que te hayan dado una canica grande, podemos eliminar algunas de estas posibilidades. En particular, no te pueden haber dado una pequeña canica brillante o una pequeña canica sin brillo.

Es igualmente probable que te hayan dado alguna de las canicas restantes, porque no conoces ninguna otra información. Así que aquí hay una serie de iconos de esas posibilidades restantes:

In [ ]:
# Just run this cell.
display_grouped_icon_array(marbles.where("size", "large").group("surface"), "large marble(s)")

**Pregunta 2.1.1.** ¿Cuál es la probabilidad de que Samantha te haya dado una canica brillante, sabiendo que te dio una canica grande?

*Sugerencia: eche un vistazo a la matriz de iconos.*


In [ ]:
probability_shiny_given_large = ...

In [ ]:
grader.check("q2_1_1")

Deberías haber descubierto que esto es diferente de la probabilidad de que Samantha te diera una canica brillante, sin tener información sobre el tamaño, que calculaste anteriormente. La distribución de superficies entre los mármoles grandes es un poco diferente a la distribución de superficies entre todos los mármoles.

In [ ]:
# Run this cell to display the icon array. Then fill answer the next question.
display_grouped_icon_array(marbles.where("surface", "shiny").group("size"), "shiny marble(s)")

**Pregunta 2.1.2.** Supongamos que Samantha hubiera dicho que te dio una canica **brillante** (¡hurra!). ¿Cuál es la probabilidad de que la canica que te dieron sea grande?

Ejecute la celda de código de arriba para mostrar la matriz de iconos, luego asigne `probability_large_given_shiny` al valor apropiado.

In [ ]:
probability_large_given_shiny = ...

In [ ]:
grader.check("q2_1_2")

**Pregunta 2.1.3.** ¿Puedes responder las dos preguntas anteriores simplemente mirando la matriz completa de íconos? (Puede ejecutar la celda a continuación para verla nuevamente).

*Pista:* Consulte el [Capítulo 18.2](https://inferentialthinking.com/chapters/18/2/Making_Decisions.html) para repasar cómo utilizar la regla de Bayes.

In [ ]:
# Just run this cell.  The next cell is where you should write your answer.
display_grouped_icon_array(marbles.group(["surface", "size"]), "marble(s)")

Si puedes, ¿cómo? ¿Si no, porque no? Consulte con sus compañeros de laboratorio o con un miembro del personal para ver si está en el camino correcto.


_Escriba su respuesta aquí, reemplazando este texto._

# 3. Detección de cáncer
Esperamos que las matrices de iconos de la parte anterior le hayan ayudado a desarrollar una intuición sobre por qué las probabilidades condicionales pueden ser útiles. Ahora, veamos una aplicación de la vida real.

### Fondo
Las pruebas médicas son muy importantes, especialmente para el cáncer. Una prueba de detección de cáncer básica implica buscar el cáncer antes de que una persona presente algún síntoma.

La detección del cáncer no es 100% confiable y puede tener errores. Hay casos en los que alguien sin cáncer o con un tumor benigno ve un resultado positivo en la prueba. También hay casos en los que alguien con cáncer puede recibir un resultado negativo. (“Positivo” implica una indicación de cáncer en este contexto). El primer caso, llamado falso positivo, podría causar ansiedad al paciente y dar lugar a pruebas adicionales que pueden ser riesgosas e implicar radiación innecesaria. En el segundo caso, llamado falso negativo, una persona no recibiría la información necesaria y el posible tratamiento para su condición.

La probabilidad condicional puede proporcionar información sobre la precisión de las pruebas iniciales de cáncer. Por ejemplo, se puede calcular la probabilidad de que una persona tenga cáncer según los resultados de una prueba de diagnóstico combinando información de diferentes distribuciones de probabilidad. Verá que la probabilidad de que la Persona X tenga cáncer incluso si recibe un resultado positivo no es necesariamente del 100%.

Es importante señalar que las probabilidades condicionales ofrecen sólo una línea de información para comprender el resultado de una prueba y que este enfoque estadístico no tiene en cuenta factores importantes como la herencia o el entorno que llegan a afectar la salud y el diagnóstico de una persona. Los [Centros para el Control y la Prevención de Enfermedades] (https://www.cdc.gov/cancer/dcpc/prevention/screening.htm) (CDC) recomiendan realizarse exámenes de detección de cáncer preventivos y siempre se debe consultar a un médico si tiene dudas. sobre su salud.

## 3.1. Estadísticas básicas sobre el cáncer

Nota: Todas las siguientes estadísticas son inventadas y no reflejan necesariamente el estado real del mundo.


Supongamos que en un grupo representativo de 10.000 personas a las que se les hacen pruebas de cáncer ("representativo", lo que significa que las frecuencias de diferentes eventos son las mismas que las frecuencias en toda la población):
1. 100 personas tienen cáncer.
2. Entre las 100 personas que tienen cáncer, 90 tienen resultados positivos en una prueba de cáncer y 10 tienen resultados negativos. (Entonces 10 personas reciben resultados falsos negativos).
3. Las otras 9.900 personas no tienen cáncer.
4. Entre estas 9.900 personas, 198 tienen resultados positivos en una prueba de cáncer y las otras 9.702 tienen resultados negativos. (Entonces 198 ven resultados "falsos positivos").

A continuación hemos generado una tabla con datos de estas 10.000 personas hipotéticas.

*Nota: Estas estadísticas son inventadas y no reflejan necesariamente el estado real del mundo.*

In [ ]:
people = Table().with_columns(
    "status", ["cancer", "cancer", "no cancer", "no cancer"],
    "test status", ["positive", "negative", "positive", "negative"],
    "count", [90, 10, 198, 9702])
people

Una forma de visualizar este conjunto de datos es con una tabla de contingencia, que ya ha visto antes.

**Pregunta 3.1.1.** Utilizando la tabla "personas" definida anteriormente, cree una tabla de contingencia similar a esta:

|estado|negativo|positivo|
|-|-|-|
|cáncer|||
|sin cáncer||||

con el **recuento** de cada grupo completado, según lo que te hemos contado anteriormente. Los conteos en las 4 casillas deben sumar 10,000.


In [ ]:
cancer = ...
cancer

In [ ]:
grader.check("q3_1_1")

Aquí están los datos de "personas" en una matriz de iconos.

In [ ]:
display_grouped_icon_array(people, "people who've taken a cancer test")

Ahora pensemos en cómo se puede utilizar este tipo de información cuando a la Persona X se le haga una prueba de cáncer.

Antes de conocer cualquier información sobre la Persona X, podría imaginarse a la Persona X como una **muestra aleatoria uniforme** de una de las 10,000 personas en esta población imaginaria de personas que han sido examinadas.

¿Cuál es la probabilidad de que la Persona X tenga cáncer sin saber nada más sobre ella? Es $\frac{100}{10000}$, o 1%. Podemos ver eso más directamente con esta matriz de iconos:

In [ ]:
by_health = people.select(0, 2).group(0, sum).relabeled(1, 'count')
display_grouped_icon_array(by_health, "people who've taken a cancer test")

**Pregunta 3.1.2.** ¿Cuál es la probabilidad de que la Persona X tenga un resultado positivo en la prueba sin saber nada más sobre ella?
Ejecute la siguiente celda de código para mostrar una matriz de iconos, luego asigne `probability_positive_test` a este valor.

In [ ]:
# Run this cell first to display an icon array. Then fill in the probability of a positive test result in the last line.
by_test = people.select(1, 2).group(0, sum).relabeled(1, 'count')
display_grouped_icon_array(by_test, "people who've taken a cancer test")

# Now fill in the probability of a positive test result
probability_positive_test = ...

In [ ]:
grader.check("q3_1_2")

## 3.2. Interpretación de los resultados de las pruebas
Supongamos que la Persona X tiene un resultado positivo en la prueba. Esto significa que ahora puedes limitarlos a formar parte de solo uno de los dos grupos siguientes:
1. Las personas con cáncer que tengan un resultado positivo en la prueba.
2. Las personas sin cáncer que tengan un resultado positivo en la prueba.

Aquí hay una matriz de iconos para esos dos grupos:

In [ ]:
# Just run this cell.
display_grouped_icon_array(people.where("test status", are.equal_to("positive")).drop(1), "people who have a positive test result")

La *probabilidad condicional* de que la Persona X **tenga cáncer dado el resultado positivo de su prueba** es la posibilidad de que esté en el primer grupo (cáncer), suponiendo que tenga un resultado positivo de la prueba.

**Pregunta 3.2.1.** Si observamos la matriz de iconos de arriba, la probabilidad condicional de que la Persona X tenga cáncer **dado el resultado positivo de su prueba** es la más cercana a:

1. $\displaystyle\frac{9}{10}$

2. $\displaystyle\frac{2}{3}$

3. $\displaystyle\frac{1}{2}$

4. $\displaystyle\frac{1}{3}$

5. $\displaystyle\frac{1}{100}$


Asigne `rough_prob_cancer_given_positive` a un número entero correspondiente a su respuesta.

In [ ]:
# Set this to either 1, 2, 3, 4, or 5 corresponding to the correct probability.
rough_prob_cancer_given_positive = ...

In [ ]:
grader.check("q3_2_1")

**Pregunta 3.2.2.** Ahora escribe código para calcular esa probabilidad exactamente, usando la tabla de contingencia original que escribiste (la tabla de "cáncer").

Ejecute la siguiente celda de código para ver la tabla "cáncer", luego complete "prob_cancer_given_positive" con su código.



In [ ]:
# Run this cell first to see the cancer table. Then fill in the next line of this cell.
cancer.show()

prob_cancer_given_positive = ...

print('Probability of cancer given positive test result: {}'.format(prob_cancer_given_positive))

In [ ]:
grader.check("q3_2_2")

**Pregunta 3.2.3.** Mire nuevamente la matriz completa de íconos. Usando eso, ¿cómo calcularías la probabilidad condicional de cáncer si la prueba fuera positiva?

Ejecute la siguiente celda de código para ver la matriz de iconos completa.


_Escriba su respuesta aquí, reemplazando este texto._

In [ ]:
# The full icon array is given here for your convenience.
# Write your answer in the previous cell.
display_grouped_icon_array(people, "people who've taken a cancer test")

**Pregunta 3.2.4.** ¿Su respuesta a la pregunta 3.2.2 es mayor que la proporción general de personas en la población que tienen cáncer (indicada como 1 % en 3.1.1)? ¿Tiene sentido? Consulte con sus compañeros o con un miembro del personal para ver si tiene la idea correcta.

_Escriba su respuesta aquí, reemplazando este texto._

# 4. Un previo subjetivo

Tener razón no siempre es satisfactorio. Clasificar a un paciente positivo como si no tuviera cáncer todavía parece algo incorrecto, para una prueba tan precisa. En esta sección, analizaremos en mayor profundidad la base de nuestro cálculo de probabilidad: el supuesto de aleatoriedad.

Nuestra suposición era que se hizo la prueba a una persona elegida al azar de nuestra población de 10.000 individuos. Esto no sucede en la realidad. Las personas acuden a hacerse la prueba porque creen que podrían tener la enfermedad o porque su médico cree que podrían tener la enfermedad. Las personas que se hacen la prueba no son miembros de la población elegidos al azar.

Es por eso que nuestra intuición sobre las personas que se hacían la prueba no encajaba bien con la respuesta que obtuvimos. En la **Pregunta 3.2.2**, calculamos que la probabilidad de cáncer si el resultado de la prueba era positivo era **0,3125**, o **31,25%**. Esto parece inusualmente bajo, especialmente teniendo en cuenta lo que sabemos sobre la eficacia de las pruebas de detección del cáncer. Estábamos imaginando una situación realista en la que un paciente iba a hacerse la prueba porque había alguna razón para hacerlo, mientras que el cálculo se basaba en una persona elegida al azar para la prueba.

Así que rehagamos nuestro cálculo bajo la suposición más realista de que el paciente se hace la prueba porque el médico cree que existe la posibilidad de que tenga la enfermedad y comparemos las dos probabilidades.

**Aparte:** Para obtener más información sobre la Parte 4 de esta práctica de laboratorio, consulte la [Sección 18.2.2](https://inferentialthinking.com/chapters/18/2/Making_Decisions.html#a-subjective-prior) de el libro de texto.

## 4.1. Una nueva mirada a la población
Supongamos ahora que nuestra población de 10.000 personas tiene en cuenta el hecho de que la elección de realizarse un examen médico no se hace al azar; debe haber algún motivo para que un individuo tome uno. Así, nuestra población modificada tiene las siguientes propiedades:
1. 1.000 personas tienen cáncer.
1. Entre las 1.000 personas que tienen cáncer, 900 tienen resultados positivos en una prueba de cáncer y 100 tienen resultados negativos.
3. Las otras 9.000 personas no tienen cáncer.
4. Entre estas 9.000 personas, 180 tienen resultados positivos en una prueba de cáncer y las otras 8.820 tienen resultados negativos. (Entonces 180 ven resultados "falsos positivos").

A continuación hemos generado una tabla de población llamada `people_new` y una tabla de contingencia con datos de estas 10.000 personas hipotéticas llamada `cancer_new`.

In [ ]:
# Just run this cell to load the table
people_new = Table().with_columns(
    "status", ["cancer", "cancer", "no cancer", "no cancer"],
    "test status", ["positive", "negative", "positive", "negative"],
    "count", [900, 100, 180, 8820])
people_new

In [ ]:
# Just run this cell to load the table
cancer_new = Table().with_columns(
    "status", ["cancer", "no cancer"],
    "negative", [100, 8820],
    "positive", [900, 180])
cancer_new

Como antes, podemos presentar los datos anteriores en una matriz de iconos. Aquí están los datos `people_new` en dicho formato.

In [ ]:
display_grouped_icon_array(people_new, "people who've taken a cancer test")

## 4.2. Interpretación y comparación
Como en la Parte 3, supongamos que la Persona X tiene un resultado positivo en la prueba. Esto significa que ahora puedes limitarlos a formar parte de solo uno de los dos grupos siguientes:
1. Las personas con cáncer que tengan un resultado positivo en la prueba.
2. Las personas sin cáncer que tengan un resultado positivo en la prueba.

Tal como lo hicimos en la Parte 3 de esta práctica de laboratorio, aquí hay una matriz de íconos para esos dos grupos:

In [ ]:
# Just run this cell.
display_grouped_icon_array(people_new.where("test status", are.equal_to("positive")).drop(1), "people who have a positive test result")

**Pregunta 4.2.1.** Utilizando la nueva tabla de contingencia, `cancer_new`, que se proporcionó al principio de esta sección, escriba código para calcular la probabilidad exacta de que un individuo tenga cáncer dado que ha recibido una prueba positiva. resultado.

Ejecute la siguiente celda de código para ver la tabla `cancer_new`, luego complete `prob_cancer_given_positive_new` con su código.


In [ ]:
# Run this cell first to see the cancer_new table. Then fill in the next line of this cell.
cancer_new.show()

prob_cancer_given_positive_new = ...

print(f'Probability of cancer given positive test result: {prob_cancer_given_positive_new}')

In [ ]:
grader.check("q4_2_1")

**Pregunta 4.2.2.** ¿Cómo se compara su probabilidad en 3.2.2 con su respuesta de 4.2.1? ¿Tiene sentido? Consulte con sus compañeros o con un miembro del personal para ver si tiene la idea correcta.

_Escriba su respuesta aquí, reemplazando este texto._

## ¡Hecho!

<img src="https://github.com/darioblandon/materials-fa23/blob/main/lab/lab10/yogi.jpeg?raw=1" alt="Photo of dog on white carpet with phone nearby." width="250"/>

¡Felicitaciones, ya terminó con _todos_ los laboratorios de Data 8! **¡Yogi** está muy orgulloso de ti!

## Envío

Asegúrese de haber ejecutado todas las celdas de su cuaderno en orden antes de ejecutar la siguiente celda, para que todas las imágenes/gráficos aparezcan en el resultado. La siguiente celda generará un archivo zip para que lo envíe. **¡Guarde antes de exportar!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)